# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [165]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests, json
# Import API key
from api_keys import geoapify_key

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [58]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities_beta.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ushuaia,-54.8061,-68.3185,283.98,82,75,7.20,AR,1675013072
1,1,rikitea,-23.1230,-134.9680,299.00,74,77,4.86,PF,1675013073
2,2,hermanus,-34.4175,19.2361,293.62,59,100,10.86,ZA,1675013073
3,3,yabrud,33.9665,36.6610,279.20,69,72,0.42,SY,1675013074
4,4,tasiilaq,65.6153,-37.6378,263.43,79,32,2.32,GL,1675013074


In [67]:
# Configure the map plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [6]:
map_plot = city_data_df.hvplot.points(
    "Lat",
    "Lng",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.01,
    color = "City"
)

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [68]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles= "OSM", 
    frame_width = 800,
    frame_height = 600,
    color = "City",
    size = "Humidity",

)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [72]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_cities_df = city_data_df.loc[city_data_df["Humidity"] <= 100,:]
narrow_cities_df.head()

# Drop any rows with null values
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ushuaia,-54.8061,-68.3185,283.98,82,75,7.20,AR,1675013072
1,1,rikitea,-23.1230,-134.9680,299.00,74,77,4.86,PF,1675013073
2,2,hermanus,-34.4175,19.2361,293.62,59,100,10.86,ZA,1675013073
3,3,yabrud,33.9665,36.6610,279.20,69,72,0.42,SY,1675013074
4,4,tasiilaq,65.6153,-37.6378,263.43,79,32,2.32,GL,1675013074


### Step 3: Create a new DataFrame called `hotel_df`.

In [104]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow_cities_df[["City","Country","Lat","Lng","Humidity"]].copy(deep=True)
hotel_df.head()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,ushuaia,AR,-54.8061,-68.3185,82,
1,rikitea,PF,-23.1230,-134.9680,74,
2,hermanus,ZA,-34.4175,19.2361,59,
3,yabrud,SY,33.9665,36.6610,69,
4,tasiilaq,GL,65.6153,-37.6378,79,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [158]:
import requests

url = "https://api.geoapify.com/v2/places?"

# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "bias": f"proximity:{hotel_df['Lng'][0]},{hotel_df['Lat'][0]}",
    "limit": 1,
    "apiKey": geoapify_key
}

params["filter"] = f"circle:{hotel_df['Lng'][0]},{hotel_df['Lat'][0]},{radius}"

#response = requests.get(url,params).json()
#print(response["features"][0]["properties"]["name"])
#print(json.dumps(response, indent=4, sort_keys= True))

{'categories': 'accommodation.hotel', 'bias': 'proximity:-68.3185,-54.8061', 'limit': 1, 'apiKey': '1cb88fe8bd93490dabd9f79b82c6a59e', 'filter': 'circle:-68.3185,-54.8061,10000'}


In [162]:
for index, row in hotel_df.iterrows():
    print(row[3])
    print(row[2])
    print(index)
    # get latitude, longitude from the DataFrame

-68.3185
-54.8061
0
-134.968
-23.123
1
19.2361
-34.4175
2
36.661
33.9665
3
-37.6378
65.6153
4
-114.3774
62.4541
5
28.9218
-30.8249
6
73.3876
1.7959
7
152.1441
-32.7218
8
167.6908
-46.1323
9


In [163]:
import requests

# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    long_tude = row[3]
    lati_tude = row[2]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{long_tude},{lati_tude},{radius}"
    params["bias"] = f"proximity:{long_tude},{lati_tude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(url,params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ushuaia - nearest hotel: Hostal Paisaje del Beagle
rikitea - nearest hotel: Chez Bianca & Benoit
hermanus - nearest hotel: Aloe guest house
yabrud - nearest hotel: No hotel found
tasiilaq - nearest hotel: Angmagssalik
yellowknife - nearest hotel: Gold Range Hotel
umzimvubu - nearest hotel: No hotel found
hithadhoo - nearest hotel: No hotel found
nelson bay - nearest hotel: No hotel found
tuatapere - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
0,ushuaia,AR,-54.8061,-68.3185,82,Hostal Paisaje del Beagle
1,rikitea,PF,-23.1230,-134.9680,74,Chez Bianca & Benoit
2,hermanus,ZA,-34.4175,19.2361,59,Aloe guest house
3,yabrud,SY,33.9665,36.6610,69,No hotel found
4,tasiilaq,GL,65.6153,-37.6378,79,Angmagssalik
5,yellowknife,CA,62.4541,-114.3774,75,Gold Range Hotel
6,umzimvubu,ZA,-30.8249,28.9218,93,No hotel found
7,hithadhoo,MV,1.7959,73.3876,74,No hotel found
8,nelson bay,AU,-32.7218,152.1441,97,No hotel found
9,tuatapere,NZ,-46.1323,167.6908,63,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [170]:
#%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles= "OSM", 
    frame_width = 800,
    frame_height = 600,
    color = "City",
    size = "Humidity",
    hover_cols="all"    

)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,index,Country,Hotel Name)